### Kipoi usage tutorial: Combined Predictions for a Patient

This tutorial shows how to apply the model to a vcf file, if you would like to predict the *combined* effect of variants in a *single* patient.

For this you need a (single-sample) vcf with the variants of this particular patient.

The combined effect is interesting to check if a patient has several variants in the same 5'UTR that cooperate to lead to a strong variant effect. E.g. if your patient has two variants in the 5'UTR of a specific gene, which combine to form a new uTIS, this approach will detect that.

In [1]:
# Imports
import kipoi
import os
import numpy as np
import pandas as pd

### Test run
Make a test run to check that the environment is correctly configured and all required packages work correctly

In [2]:
!kipoi test "../Framepool"

OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 27,31,59,63
OMP: Info #214: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #156: KMP_AFFINITY: 4 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #285: KMP_AFFINITY: topology layer "LL cache" is equivalent to "socket".
OMP: Info #285: KMP_AFFINITY: topology layer "L3 cache" is equivalent to "socket".
OMP: Info #285: KMP_AFFINITY: topology layer "L2 cache" is equivalent to "core".
OMP: Info #285: KMP_AFFINITY: topology layer "L1 cache" is equivalent to "core".
OMP: Info #191: KMP_AFFINITY: 1 socket x 2 cores/socket x 2 threads/core (2 total cores)
OMP: Info #216: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #171: KMP_AFFINITY: OS proc 27 maps to socket 3 core 6 thread 0 
OMP: Info #171: KMP_AFFINITY: OS proc 59 maps to socket 3 core 6 thread 1 
OMP: Info #171: KMP_AFFINITY: OS proc 31 maps to socket 3 core 7 thread 0 
OMP: Info #171: KMP_AFFINITY: OS proc 63 maps to socket 3 core 7 thread 1 

### Load the model

In [3]:
# Source model directly from directory
model = kipoi.get_model("../Framepool", source="dir")

Using downloaded and verified file: /data/nasif12/home_if12/karollus/projects/models/Framepool/downloaded/model_files/weights/d1e9656725e730d509a09d5371e51bd2
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


### Optional: download example files and hg19 fasta

In case you want to run the example, you can download the example files using this code.
If you have your own files, you can skip this step and simply adjust the paths in the next step

In [4]:
import urllib.request
import gzip
import shutil

In [5]:
# make ExampleFile directory if it does not exist
if not os.path.exists("ExampleFiles"):
    os.makedirs("ExampleFiles")

In [6]:
# Download vcf
urllib.request.urlretrieve("https://zenodo.org/record/3584238/files/patho.vcf.gz?download=1", 'ExampleFiles/patho.vcf.gz')
# Download vcf tabix
urllib.request.urlretrieve("https://zenodo.org/record/3584238/files/patho.vcf.gz.tbi?download=1", 'ExampleFiles/patho.vcf.gz.tbi')
# Download bed (with prefix)
urllib.request.urlretrieve("https://zenodo.org/record/3584238/files/gencodev19_5utr_sorted.bed?download=1", 'ExampleFiles/gencodev19_5utr_sorted.bed')
# Download chromosome order file (with prefix)
urllib.request.urlretrieve("https://zenodo.org/record/3584238/files/chrom_order.txt?download=1", 'ExampleFiles/chrom_order.txt')
# Download id mapping file
urllib.request.urlretrieve("https://zenodo.org/record/3584238/files/hg19_idmap.tsv?download=1", 'ExampleFiles/hg19_idmap.tsv')

('ExampleFiles/hg19_idmap.tsv', <http.client.HTTPMessage at 0x2b6c9e5a0ac8>)

In [7]:
# Download gzipped hg19 fasta (warning: 900mb)
urllib.request.urlretrieve("https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz", 'ExampleFiles/hg19.fa.gz')
# unzip
with gzip.open('ExampleFiles/hg19.fa.gz', 'rb') as f_in:
    with open('ExampleFiles/hg19.fa', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

### Provide the parameters
This Dataloader requires the following input files:
1. bed3+ where a specific user-specified column (>3, 1-based) of the bed denotes the strand
   and a specific user-specified column (>3, 1-based) of the bed denotes the transcript id
   (or some other id that explains which exons in the bed belong together to form one sequence).
   All columns of the bed, except the first three, the id and the strand, are ignored. 
2. fasta file that provides the reference genome
3. bgzip compressed (single sample) vcf that provides the variants
4. A chromosome order file (such as a fai file) that specifies the order of chromosomes 
   (must be valid for all files)
   
The bed and vcf must both be sorted (by position) and a tabix index must be present (must lie in the same directory and have the same name + .tbi)

The num_chr flag indicates whether chromosomes are listed numerically or with a chr prefix. This must be consistent across all input files!

**NB: Some common issues**
1. Files do not agree with respect to the ordering or the chr prefix. This creates all kinds of problems.
2. The vcf includes non-DNA characters, e.g. * or /. The model cannot currently account for this
3. A multi-sample vcf is used, or one which mixes several individuals. This creates nonsensical results, since all variants that intersect a given 5'UTR are simultenously injected. Thus the predicted effects will not reflect the reality of one patient, but a mix of all of them. Split the vcf by patient and predict seperately for each
4. There is more than one alternative allele for a given position in the vcf. In this case, currently, only the first is used, and the rest are ignored (as otherwise it is unclear what the variant effect is supposed to be). As a workaround, you can split into several vcf.

In [8]:
# Path of the vcf file
vcf_path = "ExampleFiles/patho.vcf.gz"

# Path of the fasta file
fasta_path = "ExampleFiles/hg19.fa"
# Set true if fasta has no chr prefix, false otherwise
num_chr = False

# Path of the bed file specifying human 5utr
# For gencode 19, if chr prefix is present, use:
# gencodev19_5utr_sorted.bed
# Else: gencodev19_5utr_sorted_noprefix.bed (can be found in same zenodo repository)
bed_path = "ExampleFiles/gencodev19_5utr_sorted.bed"
id_column = 4

# chromosome order file (the example file has the natural order)
# a noprefix version can also be found at the Zenodo repository
chr_order_path = "ExampleFiles/chrom_order.txt"

# output file path
output_file_path = "patho.tsv"

### Run Prediction

In [9]:
model.pipeline.predict_to_file(output_file_path, {"intervals_file":bed_path, 
                               "fasta_file":fasta_path,
                               "vcf_file":vcf_path,
                               "chr_order_file": chr_order_path,
                               "id_column":id_column,
                               "num_chr":num_chr},
                              batch_size=64)

100%|██████████| 1/1 [01:16<00:00, 76.46s/it]


### Load results

In [10]:
# Load data as dataframe
df = pd.read_csv(output_file_path, sep="\t")
df = df.rename(index=str, columns={"metadata/chr":"chr",
          "metadata/exon_positions":"exon_positions",
          "metadata/id":"transcript_id",
          "metadata/strand":"strand",
          "metadata/variants":"variants",
          "preds/mrl_fold_change":"mrl_fold_change",
          "preds/shift_1":"shift_1",
          "preds/shift_2":"shift_2"}
)

### Optional: id map to get a clearer output

In [11]:
# id map path
# Provide an id mapping file to get a richer output:
id_map_path = "ExampleFiles/hg19_idmap.tsv"

# Id map
df_map = pd.read_csv(id_map_path, sep="\t")
df = df.merge(df_map, on="transcript_id")

### Show results

Each row of the results table corresponds to a transcript whose 5'UTR region (defined in the bed) intersects at least one variant in the vcf
The table provides the following information:
- exon_positions: the 5'UTR exons defining the 5'UTR of a specific transcript
- transcript_id: the transcript id 
- strand
- variants: the variants which were injected into this specific 5'UTR
- mrl_fold_change: the predicted log2 fold change in mean ribosome load due to the injected variants
- shift1: what the fold change would be if the canonical frame is shifted by one nt
- shift2: what the fold cange would be if the canonical frame is shifted by two nt

(if the mrl_fold_change has low magnitude, but one of the shifts has high magnitude, this indicates the creation/destruction of an in-frame uTIS that could lengthen/shorten the canonical protein)

In [12]:
df

,chr,exon_positions,transcript_id,strand,variants,mrl_fold_change,shift_1,shift_2,gene_id,gene_name
0,chr7,19156944-19157295,ENST00000242261,-,chr7:19157207:G>T;chr7:19157225:C>A,-0.789764,-0.395907,-0.773174,ENSG00000122691,TWIST1
1,chr2,96931119-96931250;96931606-96931732,ENST00000258439,-,chr2:96931137:G>A,-1.285575,-1.012779,0.028387,ENSG00000135956,TMEM127
2,chr3,98312348-98312567,ENST00000264193,-,chr3:98312358:C>T,-1.006342,-0.834495,0.085337,ENSG00000080819,CPOX
3,chr5,147211140-147211349,ENST00000296695,-,chr5:147211193:G>A,-0.361474,-0.218929,-0.283831,ENSG00000164266,SPINK1
4,chr9,21974826-21975097,ENST00000304494,-,chr9:21974860:C>A,-0.910533,-0.525239,-1.060424,ENSG00000147889,CDKN2A
5,chr11,5248251-5248301,ENST00000335295,-,chr11:5248280:C>T,-0.828528,0.006580,-0.919977,ENSG00000244734,HBB
6,chr1,209974758-209974761;209975316-209975388;209979...,ENST00000367021,-,chr1:209975361:T>A,-1.068000,-0.861996,0.055964,ENSG00000117595,IRF6
7,chr1,93297581-93297671,ENST00000370321,+,chr1:93297626:C>A,-0.799757,-0.670084,0.025305,ENSG00000122406,RPL5
8,chr17,66508542-66508720;66511534-66511540,ENST00000392711,+,chr17:66508599:G>A,-1.025860,0.015113,-1.124092,ENSG00000108946,PRKAR1A
9,chr2,96931119-96931227;96931606-96931732,ENST00000432959,-,chr2:96931137:G>A,-1.029097,-0.896816,-0.570132,ENSG00000135956,TMEM127


### Filter results
1. Choose strongest result per gene (requires id mapping)
2. Reduce to absolute effect > 0.5

In [13]:
df["abs_mrl_fc"] = np.abs(df["mrl_fold_change"])
idx = df.groupby(['gene_name'])['abs_mrl_fc'].transform(max) == df['abs_mrl_fc']
df_max = df[idx]
df_max[np.abs(df_max.mrl_fold_change) > 0.5]

,chr,exon_positions,transcript_id,strand,variants,mrl_fold_change,shift_1,shift_2,gene_id,gene_name,abs_mrl_fc
0,chr7,19156944-19157295,ENST00000242261,-,chr7:19157207:G>T;chr7:19157225:C>A,-0.789764,-0.395907,-0.773174,ENSG00000122691,TWIST1,0.789764
1,chr2,96931119-96931250;96931606-96931732,ENST00000258439,-,chr2:96931137:G>A,-1.285575,-1.012779,0.028387,ENSG00000135956,TMEM127,1.285575
2,chr3,98312348-98312567,ENST00000264193,-,chr3:98312358:C>T,-1.006342,-0.834495,0.085337,ENSG00000080819,CPOX,1.006342
5,chr11,5248251-5248301,ENST00000335295,-,chr11:5248280:C>T,-0.828528,0.006580,-0.919977,ENSG00000244734,HBB,0.828528
6,chr1,209974758-209974761;209975316-209975388;209979...,ENST00000367021,-,chr1:209975361:T>A,-1.068000,-0.861996,0.055964,ENSG00000117595,IRF6,1.068000
7,chr1,93297581-93297671,ENST00000370321,+,chr1:93297626:C>A,-0.799757,-0.670084,0.025305,ENSG00000122406,RPL5,0.799757
11,chr9,21974826-21974865,ENST00000498124,-,chr9:21974860:C>A,-1.020021,-0.026780,-1.134101,ENSG00000147889,CDKN2A,1.020021
12,chr5,147211140-147211198,ENST00000510027,-,chr5:147211193:G>A,-0.818441,-0.669264,-0.730059,ENSG00000164266,SPINK1,0.818441
14,chr17,66508567-66508689;66511534-66511540,ENST00000589228,+,chr17:66508599:G>A,-1.129289,-0.975691,0.034666,ENSG00000108946,PRKAR1A,1.129289
